# Домашнее задание 2. Часть 1

##### Трепаленко Александра, бкл182
*Для работы используется небольшой рассказ, потому что моему ноутбуку требуется очень много времени даже для его обработки.*
### Обработка книги с помощью mystem
Импортируем необходимые модули, читаем файл с книгой.

In [12]:
! pip install pycodestyle flake8 pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on
import json
from pymystem3 import Mystem


m = Mystem()
with open('smorodin.txt', encoding='utf-8-sig') as f:
    text = f.read()

### Время обработки и запись в файл
С помощью метода .analyze() распарсиваем текст. Результат записываем в файл формата .json. Время работы замеряем при помощи %time.

In [2]:
%time ana = m.analyze(text)
with open('mystem.json', 'w', encoding='utf-8') as f:
    json.dump(ana, f, ensure_ascii = False)

Wall time: 10min 13s


### Обработка книги через pymorphy
Импортируем новые модули. Чистим текст от пунктуации и создаем пустой список для слов.

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
nltk.download('punkt')


morph = MorphAnalyzer()
PUNCT = list(""".,…;:!?@#$%^&*№()_—=+|[]{}/\"<>`~±§«»""") + ['- ', ' -', ' - ']
for sign in PUNCT:
    text = text.replace(sign, '')
words_from_text = []

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Время обработки и запись в файл
При помощи word_tokenize и .parse() сначала токенизируем, а потом разбираем словоформы. В созданный ранее список в видео кортежа записываем лемму и часть речи. Записываем результат в файл в формате .json. Время работы опять же замеряется при помощи %time.

In [4]:
%time words = word_tokenize(text)
for word in words:
    ana = morph.parse(word)
    new_word = ana[0]
    a = (new_word.word, new_word.tag.POS, new_word.normal_form)
    words_from_text.append(a)
with open('pymorphy.json', 'w', encoding='utf-8') as f:
    json.dump(words_from_text, f, ensure_ascii = False)

Wall time: 169 ms


## Вопросы

### Какую долю слов составляет каждая часть речи?
Импоритруем модуль Counter. Создаем списки для всех частей речи и лемм. Берем каждый элемент из списка всех слов текста и отправляем его составляющие в соответствующие списки. Считаем, сколько рах встречается каждая часть речи.

In [5]:
from collections import Counter


all_POS = []
all_lemmas = []
for element in words_from_text:
    all_POS.append(element[1])
    all_lemmas.append(element[2])
c = Counter(all_POS)

### Ответ

In [6]:
print('Часть речи - процент:')
for POS, number in c.items():
    freq = str(number*100/len(all_POS))
    print(str(POS) + ' - ' + freq + '%')

Часть речи - процент:
NOUN - 27.5%
ADJF - 9.705882352941176%
ADJS - 2.6715686274509802%
VERB - 14.215686274509803%
PREP - 9.044117647058824%
NPRO - 9.485294117647058%
PRTF - 0.46568627450980393%
ADVB - 5.857843137254902%
CONJ - 10.073529411764707%
PRCL - 5.857843137254902%
INTJ - 0.29411764705882354%
PRTS - 0.6617647058823529%
GRND - 0.5392156862745098%
INFN - 1.7156862745098038%
PRED - 0.6372549019607843%
NUMR - 1.0049019607843137%
COMP - 0.2696078431372549%


### Топ-20 глаголов и наречий
Создаем списки для глаголов и наречий. Для каждого элемента списка слов проверяем, является ли он глаголом или наречием, и отправляем в соответствующий список. При помощи метода most_common оставляем самые частотные варианты.

In [7]:
verbs = []
adverbs = []
for element in words_from_text:
    if element[1] == 'VERB':
        verbs.append(element[2])
    if element[1] == 'ADVB':
        adverbs.append(element[2])
c_verbs = Counter(verbs)
c_adverbs = Counter(adverbs)
common_verbs = c_verbs.most_common(20)
common_adverbs = c_adverbs.most_common(20)

### Ответ

In [9]:
print('Топ-20 глаголов\nГлагол - количество:')
for verb in common_verbs:
    print(verb[0], '-', verb[1])
print('\nТоп-20 наречий\nНаречие - количество:')
for adverb in common_adverbs:
    print(adverb[0], '-', adverb[1])

Топ-20 глаголов
Глагол - количество:
быть - 86
сказать - 20
мочь - 13
видеть - 12
любить - 9
заказывать - 9
произнести - 7
есть - 7
хотеть - 7
проговорить - 6
спросить - 6
жить - 6
думать - 6
ответить - 6
помнить - 5
понимать - 5
принести - 5
ужинать - 5
ходить - 5
находить - 5

Топ-20 наречий
Наречие - количество:
весьма - 12
много - 10
там - 8
здесь - 8
всегда - 7
очень - 7
назад - 6
вполне - 5
слишком - 5
уже - 5
совершенно - 5
никогда - 4
где - 4
сюда - 4
естественно - 4
хорошо - 4
снова - 4
действительно - 4
утром - 4
вовсе - 4


### Топ-25 биграмм и триграмм
При помощи ngrams собираем список сначала биграмм, затем триграмм. Для каждого используем Counter и оставляем 25 наиболее частотных вариантов.

In [10]:
from nltk.util import ngrams


bigrams = list(ngrams(all_lemmas, 2))
c_b = Counter(bigrams)
common_bigrams = c_b.most_common(25)
trigrams = list(ngrams(all_lemmas, 3))
c_t = Counter(trigrams)
common_trigrams = c_t.most_common(25)

### Ответ

In [11]:
print('Биграмма - количество:')
for bigram in common_bigrams:
    print(*bigram[0], '-', bigram[1])
print('\nТриграмма - количество')
for trigram in common_trigrams:
    print(*trigram[0], '-', trigram[1])

Биграмма - количество:
эркюля пуарый - 20
он быть - 15
чёрный смородина - 14
что он - 12
мистер бонингтон - 11
это быть - 11
мистер гаскон - 11
я не - 10
в ресторан - 9
он не - 8
и он - 8
у он - 8
миссис хилый - 8
тот же - 7
в тот - 6
пирог с - 6
не быть - 6
генри гаскон - 6
доктор мак-эндрю - 6
джордж рамзь - 6
в семь - 6
гэлант эндивор - 5
с он - 5
что я - 5
совершенно верно - 5

Триграмма - количество
у он быть - 7
в ресторан гэлант - 4
ресторан гэлант эндивор - 4
вторник и четверг - 4
пудинг с почка - 4
с почка и - 4
и пирог с - 4
пирог с чёрный - 4
с чёрный смородина - 4
в карман халат - 4
в семь тридцать - 4
год тот назад - 4
к тот же - 3
сказать эркюля пуарый - 3
сказать мистер бонингтон - 3
и чёрный смородина - 3
почка и пирог - 3
не замечать что - 3
замечать что заказывать - 3
он не быть - 3
по это повод - 3
на кингз роуд - 3
в тот же - 3
на кингстон хилла - 3
смерть мистер гаскон - 3
